[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Train YOLO11 Object Detection on a Custom Dataset

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

YOLO11 builds on the advancements introduced in YOLOv9 and YOLOv10 earlier this year, incorporating improved architectural designs, enhanced feature extraction techniques, and optimized training methods.

YOLO11m achieves a higher mean mAP score on the COCO dataset while using 22% fewer parameters than YOLOv8m, making it computationally lighter without sacrificing performance.

YOLOv11 is available in 5 different sizes, ranging from `2.6M` to `56.9M` parameters, and capable of achieving from `39.5` to `54.7` mAP on the COCO dataset.

## Setup

### Configure API keys

To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

### Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Sun Dec 22 11:31:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLO11 via Ultralytics

In [3]:
!pip list | grep torch

torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchsummary                       1.5.1
torchvision                        0.20.1+cu121


In [4]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics
!pip install -q .

Cloning into 'ultralytics'...
remote: Enumerating objects: 45832, done.
remote: Counting objects: 100% (731/731), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 45832 (delta 583), reused 305 (delta 297), pack-reused 45101 (from 5)
Receiving objects: 100% (45832/45832), 39.13 MiB | 23.92 MiB/s, done.
Resolving deltas: 100% (33934/33934), done.
/content/ultralytics
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.53 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/112.6 GB disk)


## Fine-tune YOLO11 on custom dataset

**NOTE:** When training YOLOv11, make sure your data is located in `datasets`. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' `settings.json`. In this tutorial, we will use one of the [datasets](https://universe.roboflow.com/liangdianzhong/-qvdww) available on [Roboflow Universe](https://universe.roboflow.com/). When downloading, make sure to select the `yolov11` export format.

In [6]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("liangdianzhong")
project = rf.workspace("project-oo3i7").project("license-number-2x9cc")
version = project.version(1)
dataset = version.download("yolov11")

/content/datasets
loading Roboflow workspace...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to License-Number-1 in yolov11:: 100%|██████████| 3536/3536 [00:01<00:00, 1939.97it/s]


## Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11n.pt data={dataset.location}/data.yaml epochs=100 batch=64 imgsz=640 plots=True

/content
100% 5.35M/5.35M [00:00<00:00, 87.5MB/s]
New https://pypi.org/project/ultralytics/8.3.53 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/datasets/License-Number-1/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, c

**NOTE:** The results of the completed training are saved in `{HOME}/runs/detect/train/`. Let's examine them.

In [ ]:
!ls {HOME}/ultralytics/runs/detect/train/

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/ultralytics/runs/detect/train/confusion_matrix.png', width=1280)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/ultralytics/runs/detect/train/results.png', width=1280)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/ultralytics/runs/detect/train/val_batch0_pred.jpg', width=1280)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/ultralytics/runs/detect/train/P_curve.png', width=1280)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/ultralytics/runs/detect/train/R_curve.png', width=1280)

## Validate fine-tuned model

In [ ]:
!yolo task=detect mode=val model={HOME}/ultralytics/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

# Save the model into pt

In [ ]:
# Save the best model weights to a specified location
best_model_path = os.path.join(HOME, 'ultralytics/runs/detect/train/weights/best.pt')
save_path = os.path.join(HOME, 'best_model.pt') # Change to your desired save location
!cp {best_model_path} {save_path}

print(f"Best model saved to: {save_path}")

# Convert pt file into ONNX Format Model saved

In [ ]:
!pip install onnx
!pip install onnxruntime
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("/content/best_model.pt")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolo11n.onnx'

In [ ]:
from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("/content/best_model.pt")

# Export the model to ONNX format
model.export(format="tflite")  # creates 'yolo11n.onnx'

# Image Testing Model

In [ ]:
!pip install easyocr
from google.colab.patches import cv2_imshow
import cv2
import easyocr
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import csv
from datetime import datetime

In [ ]:
# Initialize EasyOCR reader
reader = easyocr.Reader(['en', 'id'])  # Include 'id' for Indonesian license plates

# Create output directory for captured images
output_dir = "captured_plates"
os.makedirs(output_dir, exist_ok=True)

# Create or open the CSV file to store number plate data
csv_file = "number_plate_data.csv"
fieldnames = ["Timestamp", "Plate Number", "Image Path"]

# Check if the CSV file exists, if not create it and write headers
if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()


def predict_and_extract_text(image_path):
    # Read the image using cv2
    img = cv2.imread(image_path)

    # Perform inference using YOLOv8
    model = YOLO("/content/best_model.pt")  # Load the model here
    results = model(img)

    # Get the first detection (assuming only one license plate)
    detection = results[0]

    # Get bounding box coordinates
    if detection.boxes.xyxy.shape[0] > 0:  # Check if any boxes are detected
        x1, y1, x2, y2 = map(int, detection.boxes.xyxy[0])

        # Crop the number plate region from the image
        cropped_image = img[y1:y2, x1:x2]

        # --- Enhancements ---
        # 1. Preprocess the cropped image:
        gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        enhanced = cv2.equalizeHist(gray)  # Apply histogram equalization
        # You can add other enhancements here, like sharpening or denoising if needed

        # 2. Perform OCR with character whitelist and other parameters:
        results_ocr = reader.readtext(
            enhanced,
            allowlist='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789',  # Whitelist alphanumeric characters
            batch_size=45,
            detail=80,
            paragraph=False,

        )

        # Process detections and save data
        for (bbox, text, prob) in results_ocr:
            if prob > 0.8:  # Filter detections based on confidence
                (top_left, top_right, bottom_right, bottom_left) = bbox
                top_left = tuple(map(int, top_left))
                bottom_right = tuple(map(int, bottom_right))

                # Draw bounding box and annotate text on the original image
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 640, 0), 2)
                cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 640, 0), 2)

                # Save captured image and data
                timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                image_filename = f"{output_dir}/plate_{timestamp}.jpg"
                cv2.imwrite(image_filename, img)  # Save the original image with bounding box

                # Append data to CSV
                with open(csv_file, mode='a', newline='') as file:
                    writer = csv.DictWriter(file, fieldnames=fieldnames)
                    writer.writerow({"Timestamp": timestamp, "Plate Number": text, "Image Path": image_filename})

                print(f"Detected Plate: {text} | Confidence: {prob:.2f} | Saved: {image_filename}")

                # Display the original image with bounding box and text using matplotlib
                fig, ax = plt.subplots(1, 1, figsize=(10, 10))  # Adjust figsize as needed
                ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                ax.set_title("Detected License Plate (Full Image)")
                plt.show()
    else:
        print("No license plate detected in the image.")


# Example usage
image_path = '/content/Plat-H-Semarang.jpg'  # Replace with the path to your image
predict_and_extract_text(image_path)